## **WEEK 2**

- EDA
- Validation
- Data Leakage

## Exploratory Data Analysis(EDA)

### 概要

1. EDA とは何か？なぜするのか？
2. 探索するもの
3. 探索と可視化のツール
4. データセットクリーニング
5. Kaggle コンペにおける EDA

### EDAとは何か？なぜするのか？

- EDAとは
    - データをよく観察し、データをより良く理解すること
- EDAをする理由は
    - 強い特徴量を生み出し、正確なモデルを構築するため
    - データに関する直感を身につけるため
    - 新しい効果的な特徴量生成のための仮説を生み出すため
    - より良いスコアのためのインサイトを見つけるため
    
- 複雑なモデルは上位解法でよく見られ、重要ではあるが、一方で彼らは注意深くEDAを行っている。
    - モデリングに時間を割くよりも EDA は重要なプロセスである
    
- コンペはまず EDA から始めるのが良い。
    
- EDA において可視化が主要な手段の一つである。
    - 可視化によってパターンを一目で分かることができる
    
- [あるコンペ](https://www.kaggle.com/c/dunnhumbychallenge)では、モデリングなしにデータの観察のみで優勝した例がある
    - プロモーションを使用するかどうかを予測するコンペ
    - プロモーションが送られた累計と使われた累計の列があったので、ユーザーごとに累計をソートして差分を取ると分類機を使用せずとも80％程度の accuracy を出すことができる。
        - 当然にこれはデータリークと言えるが、コンペにおいては通常をこれを利用しても良いことになっている

### データへの直感を身につける

- ドメイン知識の取得
    - 問題をより深く理解するのに役立つ
- データが直感的かどうかを確認する
    - ドメイン知識と一致するかどうかを確認する
- どのようにデータが生成されたかを理解する
    - 適切なバリデーションセットを構築するために必須であるため

#### ドメイン知識の取得

- コンペには様々な領域のものがあり、しばしばドメイン知識を持っていないコンペに遭遇することがある
    - ドメインに対して深く知ることはないが、通常はどんな目的か・どんなデータを持っているか・どのように人々がベースラインを構築しようとしているかを知ることは重要である。
    - Google / Wikipedia を使ってデータの種類を理解することがまず第一歩となる
    
- データのドメインの理解・列名の理解
- データの理解後は、データが直感と一致しているかを確認する（ImpはClickより必ず大きいかとか、年齢が100歳以上なのは珍しいとか）
    - 年齢が336とかなら、タイポとして33歳か36歳に手動で訂正するのが通常良いが、念の為データ説明を読み直しておく
- 一方で、Imp が0なのに Click が3の場合などは通常のエラーであることがある
    - スクリプトから送られたデータなどには付きもののエラーでランダムではないことが多いので、これは特徴量として用いるのが良い
        - 'is_incorecct' などの列を作るなどして

- データがどのように生成されたかを知ることも重要
    - データベースからサンプリングされた場合、少数クラスがオーバーサンプリングされて抽出されている可能性がある
- データがどのように生成されたかが分かれば、適切なバリデーションセットを作ることができる。
- 学習データとテストデータが異なるアルゴリズムで生成されている場合、通常以下のような特徴が見られることが多い
    - CVとLBの連動しない
    - 各特徴量の分布が異なる
    - 各データセットの期間（日付等）の範囲が異なる

#### Reading Material for Building Intution about the Data

- https://hdinwtxpwdelvatxpgqiri.coursera-apps.org/notebooks/readonly/reading_materials/EDA_video2.ipynb

### 匿名データを探索する

- 匿名データとは？
    - なぜホストはデータを匿名化するのか
- 匿名データに対して何ができるか？
    - 特徴量の真の意味を推測し、decode してみる
    - どのような前処理が必要になるかを知るために、特徴量のデータ型を推測してみる
    
#### 匿名データとは

- 機械学習モデルを構築したいけど、データそのものを公表したくないときにデータ匿名化を行うことがある
    - テキストデータがハッシュ関数を通して暗号化する
    - テーブルデータでの列名をダミー化する
- テーブルデータの列匿名化に対して
    - 個別の特徴量に対して
        - 列の真の意味を推測する
        - 列のデータ型を推測する
    - 特徴量の関係に対して
        - 特徴量ペアの関連を見つける
        - 特徴量のグループを見つける
        
- [個別の特徴量に対してのアプローチの例](https://hdinwtxpwdelvatxpgqiri.coursera-apps.org/notebooks/readonly/reading_materials/EDA_video3_screencast.ipynb)
    - Baseline Model(RF)を構築
    - RF の Feature Importance を見て、x8 が重要度が高そうなので探索する
        - 平均と分散の確認（ここでは正規分布であることが分かった）
        - そのため、元の分布に戻したくなる（scale back）
    - value_counts() で繰り返し現れる値を確認
        - 出現する数値に偏りがありそうなことがわかった
        - 例えば、ユニークな値のみ取り出し、それをソートして、差分を取る
            - 多くの差分が同じ値になった(0.043)
            - 差分の元の正確な値が分からなくても、例えばそれを1.0と想定してみる(差分を0.043で割る)
    - 元の値をそれで割ってみる
        - 小数点以下で同じような数値(1.02468)が並ぶので、これを引いてやる
        - (train.x8 - 1.02468) / 0.043 として、これを丸め込むと良い感じの整数が出てくる
    - スケールバックしたら、再びその値の value_counts() を見てみる
        - 最小値に -1968 があったので、西暦っぽいという仮説が成り立つ（ユーザーがテキストボックスに適当に数値を入れた？）
        - 1968 足すと誕生年っぽい感じになる
        - （実際にコンペが終了した後、ホストがこの列は誕生年だということを発表した）